In [33]:
#Dependencies
import numpy as np
import pandas as pd
import datetime as dt
import re

In [34]:
os.getcwd()

'/Users/jacosta3/OneDrive - University of South Florida/Gerdau Projects/pft/datasets'

## Fetching Indicators info

In [35]:
#File directory
ser_path = "./serialized/"
path = "./merged_clean/"
#change directory to datasets
os.chdir("../datasets")

In [36]:
indicators = pd.read_excel(path+"market_indicators.xlsx",sheet_name="Monthly")

In [37]:
indicators.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124 entries, 0 to 123
Data columns (total 17 columns):
 #   Column                                               Non-Null Count  Dtype         
---  ------                                               --------------  -----         
 0   dd/mm/yy                                             124 non-null    datetime64[ns]
 1   Year                                                 124 non-null    int64         
 2   Month                                                124 non-null    object        
 3   National Architect Billing/monthly                   121 non-null    float64       
 4   Total Federal Construction/Billion $                 121 non-null    float64       
 5   Total State and Local Construction/Billion $         121 non-null    float64       
 6   US Steel Capacity Utilization                        109 non-null    float64       
 7   US Long Steel/million $ per short ton                26 non-null     float64       
 8   

## Cleaning Columns

In [38]:
def clean_columns(dataframe):
    for col in dataframe.columns:
        dataframe.rename(columns={col:re.sub(r'([a-z](?=[A-Z])|[A-Z](?=[A-Z][a-z]))', r'\1 ', col)}, inplace=True)
    dataframe.columns = dataframe.columns.str.strip().str.lower().str.replace(" ","_")
    try:
        dataframe["calendar_day"] = dataframe["calendar_day"].astype("datetime64")
    except:
        pass
    try:
        dataframe["requested_date"] = dataframe["requested_date"].astype("datetime64")
        dataframe["confirmed_date"] = dataframe["confirmed_date"].astype("datetime64")
        dataframe["material_avail_date"] = dataframe["material_avail_date"].astype("datetime64")
        dataframe["load_date"] = dataframe["load_date"].astype("datetime64")
        dataframe["plan_goods_issue_date"] = dataframe["plan_goods_issue_date"].astype("datetime64")
    except:
        pass
    try:
        dataframe["snapshot_date"] = dataframe["snapshot_date"].astype("datetime64")
    finally:
        return dataframe.columns

In [39]:
clean_columns(indicators)

Index(['dd/mm/yy', 'year', 'month', 'national_architect_billing/monthly',
       'total_federal_construction/billion_$',
       'total_state_and_local_construction/billion_$',
       'us_steel_capacity_utilization',
       'us_long_steel/million_$_per_short_ton',
       'us_domestic_hot-rolled_coil/million_$_per_short_ton',
       'msci_carbon_bar_monthly_shipments',
       'msci_carbon_bar_monthly_shipments_(canada)',
       'structural_shipments_/usa', 'structural_shipments_/canada',
       'ism_manufacturing', 'ism_non_manufacturing',
       'global_steel_utilization', 'chicago#1_scrap_bushelling'],
      dtype='object')

### Create Month & Year Columns

In [40]:
indicators['year'] = indicators['dd/mm/yy'].dt.year
indicators['month'] = indicators['dd/mm/yy'].dt.month

### Rename Columns

In [41]:
indicators.columns

Index(['dd/mm/yy', 'year', 'month', 'national_architect_billing/monthly',
       'total_federal_construction/billion_$',
       'total_state_and_local_construction/billion_$',
       'us_steel_capacity_utilization',
       'us_long_steel/million_$_per_short_ton',
       'us_domestic_hot-rolled_coil/million_$_per_short_ton',
       'msci_carbon_bar_monthly_shipments',
       'msci_carbon_bar_monthly_shipments_(canada)',
       'structural_shipments_/usa', 'structural_shipments_/canada',
       'ism_manufacturing', 'ism_non_manufacturing',
       'global_steel_utilization', 'chicago#1_scrap_bushelling'],
      dtype='object')

In [42]:
#Remane column & pick only desired ones 
indicators = indicators.rename(columns={'dd/mm/yy':'calendar_day',
                                        'total_federal_construction/billion_$':'fed_construction$B',                                                              'total_state_and_local_construction/billion_$':'state_local_construction$B',
                                        'msci_carbon_bar_monthly_shipments':'carbon_bar_ship_usa',
                                        'msci_carbon_bar_monthly_shipments_(canada)':'carbon_bar_ship_canada',
                                        'structural_shipments_/usa':'structural_ship_usa', 
                                        'structural_shipments_/canada':'structural_ship_canada',
                                        'chicago#1_scrap_bushelling':'chicago_scrap$ST'})
indicators = indicators[['calendar_day','year','month', 'global_steel_utilization','fed_construction$B',                                           'state_local_construction$B','carbon_bar_ship_usa','carbon_bar_ship_canada',                                              'structural_ship_usa','structural_ship_canada', 'ism_manufacturing', 'ism_non_manufacturing',                             'chicago_scrap$ST']]

### Selecting 2016 dates on

In [43]:
#select 2016-2020
indicators = indicators[(indicators['calendar_day'].dt.year >= 2016) & (indicators['calendar_day'] <='2020-01-01')]

In [44]:
indicators.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49 entries, 72 to 120
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   calendar_day                49 non-null     datetime64[ns]
 1   year                        49 non-null     int64         
 2   month                       49 non-null     int64         
 3   global_steel_utilization    45 non-null     float64       
 4   fed_construction$B          49 non-null     float64       
 5   state_local_construction$B  49 non-null     float64       
 6   carbon_bar_ship_usa         49 non-null     float64       
 7   carbon_bar_ship_canada      49 non-null     float64       
 8   structural_ship_usa         49 non-null     float64       
 9   structural_ship_canada      49 non-null     float64       
 10  ism_manufacturing           49 non-null     float64       
 11  ism_non_manufacturing       49 non-null     float64       

### Fill NaNs

In [45]:
#filling NaNs - assumption gradual increase 
values = {36:0.7768,37:0.7915,38:0.8063,39:0.8210}
indicators['global_steel_utilization'] = indicators['global_steel_utilization'].fillna(value=values, axis=0)

In [46]:
indicators.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49 entries, 72 to 120
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   calendar_day                49 non-null     datetime64[ns]
 1   year                        49 non-null     int64         
 2   month                       49 non-null     int64         
 3   global_steel_utilization    45 non-null     float64       
 4   fed_construction$B          49 non-null     float64       
 5   state_local_construction$B  49 non-null     float64       
 6   carbon_bar_ship_usa         49 non-null     float64       
 7   carbon_bar_ship_canada      49 non-null     float64       
 8   structural_ship_usa         49 non-null     float64       
 9   structural_ship_canada      49 non-null     float64       
 10  ism_manufacturing           49 non-null     float64       
 11  ism_non_manufacturing       49 non-null     float64       

In [47]:
indicators

,calendar_day,year,month,global_steel_utilization,fed_construction$B,state_local_construction$B,carbon_bar_ship_usa,carbon_bar_ship_canada,structural_ship_usa,structural_ship_canada,ism_manufacturing,ism_non_manufacturing,chicago_scrap$ST
72,2016-01-01,2016,1,0.6600,1583.0,16684.0,204300.0,30800.0,175300.0,38700.0,48.2,53.5,180.0
73,2016-02-01,2016,2,0.6600,1840.0,17198.0,205400.0,31000.0,187000.0,37500.0,49.5,53.4,180.0
74,2016-03-01,2016,3,0.7070,1844.0,18555.0,221200.0,31400.0,203900.0,38400.0,51.8,54.5,195.0
75,2016-04-01,2016,4,0.7050,1703.0,20187.0,204600.0,31200.0,193800.0,37400.0,50.8,55.7,245.0
76,2016-05-01,2016,5,0.7130,1888.0,21954.0,200500.0,29700.0,191440.0,36600.0,51.3,52.9,275.0
77,2016-06-01,2016,6,0.6940,1931.0,25562.0,208400.0,33200.0,206400.0,40000.0,53.2,56.5,275.0
78,2016-07-01,2016,7,0.6830,1747.0,25352.0,179300.0,27700.0,178000.0,35600.0,52.6,55.5,275.0
79,2016-08-01,2016,8,0.6850,1999.0,26663.0,206800.0,30100.0,210800.0,38300.0,49.4,51.4,265.0
80,2016-09-01,2016,9,0.6850,2405.0,25578.0,192500.0,29200.0,188800.0,37600.0,51.5,57.1,235.0
81,2016-10-01,2016,10,0.6850,1678.0,25218.0,194200.0,28500.0,191000.0,38300.0,51.9,54.8,205.0


## Save to file

In [16]:
#indicators.to_pickle(ser_path+'indicators.pkl')

## Preparing Structural df
Consider whether monthly value ingestion may be better than weekly

### Importing Structural df

In [17]:
structural = pd.read_pickle(ser_path+"structural_merged.pkl")

In [18]:
structural.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158270 entries, 0 to 158269
Data columns (total 10 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   calendar_day              158270 non-null  datetime64[ns]
 1   year                      158270 non-null  object        
 2   month                     158270 non-null  object        
 3   week                      158270 non-null  object        
 4   block                     158270 non-null  object        
 5   size                      158270 non-null  object        
 6   IN_tons                   158270 non-null  float64       
 7   SO_order_qty_sales_units  158270 non-null  float64       
 8   SH_shipment_tons          158270 non-null  float64       
 9   PR_ton                    158270 non-null  float64       
dtypes: datetime64[ns](1), float64(4), object(5)
memory usage: 12.1+ MB


In [19]:
#dropping columns to permit merging
structural = structural.drop(columns=['year','month','week'])

In [20]:
structural.columns

Index(['calendar_day', 'block', 'size', 'IN_tons', 'SO_order_qty_sales_units',
       'SH_shipment_tons', 'PR_ton'],
      dtype='object')

### Creating Weekly dataframe for merging

In [21]:
weekly = structural.groupby('block').resample('W', on='calendar_day').sum().sort_values('calendar_day').reset_index()

In [22]:
weekly.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12519 entries, 0 to 12518
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   block                     12519 non-null  object        
 1   calendar_day              12519 non-null  datetime64[ns]
 2   IN_tons                   12519 non-null  float64       
 3   SO_order_qty_sales_units  12519 non-null  float64       
 4   SH_shipment_tons          12519 non-null  float64       
 5   PR_ton                    12519 non-null  float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 587.0+ KB


In [23]:
weekly.head(10)

,block,calendar_day,IN_tons,SO_order_qty_sales_units,SH_shipment_tons,PR_ton
0,"3-1/2"" ANG",2016-01-03,207.261,0.000,0.0,0.0
1,"9"" CHN",2016-01-03,43.296,0.000,0.0,0.0
2,"8 X 4"" ANG",2016-01-03,14.112,0.000,0.0,0.0
3,"6"" CHN",2016-01-03,1857.997,0.000,0.0,0.0
4,"4 X 3"" ANG",2016-01-03,472.990,0.000,0.0,0.0
5,"5 X 3-1/2"" ANG",2016-01-03,1888.340,0.000,0.0,0.0
6,"6"" FLT",2016-01-03,1102.054,2.244,0.0,0.0
7,"8"" FLT",2016-01-03,867.865,2.244,0.0,0.0
8,"5 X 3"" ANG",2016-01-03,1172.033,0.000,0.0,0.0
9,"8"" CHN",2016-01-03,958.505,0.000,0.0,0.0


In [24]:
#recreating time columns for merging
weekly['month'] = weekly['calendar_day'].dt.month
weekly['year'] = weekly['calendar_day'].dt.year

In [25]:
#weekly data ready for merging
weekly = weekly[['calendar_day','year','month','block','IN_tons','PR_ton','SO_order_qty_sales_units','SH_shipment_tons']]
weekly.head()

,calendar_day,year,month,block,IN_tons,PR_ton,SO_order_qty_sales_units,SH_shipment_tons
0,2016-01-03,2016,1,"3-1/2"" ANG",207.261,0.0,0.0,0.0
1,2016-01-03,2016,1,"9"" CHN",43.296,0.0,0.0,0.0
2,2016-01-03,2016,1,"8 X 4"" ANG",14.112,0.0,0.0,0.0
3,2016-01-03,2016,1,"6"" CHN",1857.997,0.0,0.0,0.0
4,2016-01-03,2016,1,"4 X 3"" ANG",472.990,0.0,0.0,0.0


## Merging Structural+Indicators

In [64]:
#Merginging weekly values with indicators
struct_indicators = weekly.merge(indicators, how='left', on=['year','month'], validate='m:1')

In [65]:
#Drop duplicate column and rename main weekly calendar day to calendar_day
struct_indicators = struct_indicators.drop(columns='calendar_day_y').rename(columns={'calendar_day_x':'calendar_day'})

In [67]:
struct_indicators.sample(10)

,calendar_day,year,month,block,IN_tons,PR_ton,SO_order_qty_sales_units,SH_shipment_tons,global_steel_utilization,fed_construction$B,state_local_construction$B,carbon_bar_ship_usa,carbon_bar_ship_canada,structural_ship_usa,structural_ship_canada,ism_manufacturing,ism_non_manufacturing,chicago_scrap$ST
11429,2019-08-18,2019,8,"3 X 2"" ANG",-0.010000,0.0,0.0000,0.0,0.802,2146.0,31328.0,197600.0,31900.0,208600.0,38500.0,49.1,56.4,300.0
4631,2017-07-16,2017,7,"MC4"" CHN",591.329994,0.0,0.0000,0.0,0.738,1749.0,24408.0,205900.0,29200.0,191200.0,38200.0,56.3,53.9,350.0
11877,2019-10-13,2019,10,"12"" CHN",3295.465999,0.0,77.4965,37.0,0.782,1992.0,29142.0,199800.0,34100.0,217100.0,38500.0,48.3,54.7,220.0
3793,2017-04-09,2017,4,"9"" FLT",13.376000,0.0,0.0000,2.0,0.745,1713.0,19315.0,219100.0,28500.0,188400.0,30700.0,54.8,57.5,350.0
5174,2017-09-10,2017,9,"3/4"" SQR",-0.005000,0.0,0.0000,0.0,0.739,2271.0,25388.0,220400.0,31400.0,200100.0,39400.0,60.8,59.8,365.0
4152,2017-05-21,2017,5,"8 X 4"" ANG",213.248001,0.0,0.0000,14.0,0.735,2059.0,22013.0,241800.0,34000.0,207300.0,38700.0,54.9,56.9,350.0
5249,2017-09-17,2017,9,"2-1/2"" ANG",0.100000,0.0,0.0000,0.0,0.739,2271.0,25388.0,220400.0,31400.0,200100.0,39400.0,60.8,59.8,365.0
4759,2017-07-30,2017,7,"9"" CHN",660.822002,0.0,4.2880,9.0,0.738,1749.0,24408.0,205900.0,29200.0,191200.0,38200.0,56.3,53.9,350.0
9055,2018-11-18,2018,11,"9"" CHN",7303.689942,0.0,23.0000,14.0,0.762,1880.0,23202.0,205600.0,33300.0,189200.0,38000.0,59.3,60.7,405.0
9158,2018-12-02,2018,12,"3/16X3/4"" FLT",0.000000,0.0,0.0000,0.0,0.762,1783.0,19153.0,177300.0,21900.0,174200.0,25500.0,54.3,58.0,405.0


### Saving to formats for digestion

In [68]:
#Saving df into pickle and csv
#struct_indicators.to_csv(path+"structuralweekly_with_indicators.csv")
#struct_indicators.to_pickle(ser_path+"structuralweekly_with_indicators.pkl")